In [1]:
import os
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install kaggle
!mkdir -p ~/.kaggle
!rm -rf /content/steel/
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle competitions download -c severstal-steel-defect-detection
!7z x -bsp2 -bd severstal-steel-defect-detection.zip -o/content/steel
contents = os.listdir('/content/steel/')

100% 1.57G/1.57G [01:03<00:00, 23.8MB/s]
100% 1.57G/1.57G [01:03<00:00, 26.6MB/s]

7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan         1 file, 1684204253 bytes (1607 MiB)

Extracting archive: severstal-steel-defect-detection.zip
 22% 4096 Open              --
Path = severstal-steel-defect-detection.zip
Type = zip
Physical Size = 1684204253

  0%      1% 211 - test_images/0986968be.jpg                                      1% 392 - test_images/122dea056.jpg                                      2% 574 - test_images/1b0c8b4c1.jpg                                    

In [13]:
!pip install tensorflow==2.8.0 keras==2.8.0
!pip install segmentation-models

In [4]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import pickle
import matplotlib.patches as patches
import re
import random
from sklearn.model_selection import train_test_split
import cv2
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import plot_model
from PIL import Image
import tensorflow as tf
import keras
from keras import backend as K
from keras.models import Model,load_model
from keras.regularizers import l2
import datetime
%load_ext tensorboard
import segmentation_models
from segmentation_models import Unet
from segmentation_models import get_preprocessing
import imgaug.augmenters as iaa
import segmentation_models as sm
sm.set_framework('tf.keras')
sm.framework()
from tensorflow.keras.losses import binary_crossentropy
from tqdm import tqdm

Segmentation Models: using `keras` framework.


In [5]:
test_image=[i for i in os.listdir('/content/steel/test_images')]

In [6]:
len(test_image)

5506

## Final Pipeline



    Binary Classifier trained with all Images with 1 output probability i.e. [defect] which tells whether whether Image have defect or not while Multi-label Classifier trained only with Images with defect. Multi-label Classifier have 4 output probability i.e. [defect_1,defect_2,defect_3,defect_4].

    Segmentation models are trained on each defect separately and predicts mask for each defect. Overall,we have 4 segment models i.e. one for each defect. There are total 6 models that are saved and later uses area threshold and class probability (different for each defect, depending on particular defect mask area and class probability).

    At last Encoded Pixels for test images are predicted and submitted. While training one must take care that proper data is fed to each model in the network which will help to reduce loss.



In [7]:
def f1_score(y_true, y_pred):
    #https://aakashgoel12.medium.com/how-to-add-user-defined-function-get-f1-score-in-keras-metrics-3013f979ce0d
    #https://stackoverflow.com/questions/43547402/how-to-calculate-f1-macro-in-keras
    true_positives=K.sum(K.round(K.clip(y_true*y_pred,0,1)))   #calculates number of true positives
    possible_positives=K.sum(K.round(K.clip(y_true,0,1)))      #calculates number of actual positives
    predicted_positives=K.sum(K.round(K.clip(y_pred,0,1)))

    #K.epsilon takes care of non-zero divisions
    #was modified by adding the constant epsilon, in order to avoid division by 0. Thus NaN will not be computed.
    precision=true_positives/(predicted_positives +K.epsilon())
    recall=true_positives/(possible_positives+K.epsilon())
    f1_val=2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

In [8]:
#https://stackoverflow.com/questions/31273652/how-to-calculate-dice-coefficient-for-measuring-accuracy-of-image-segmentation-i
def dice_coef(y_true,y_pred):
    y_true_f=tf.reshape(tf.dtypes.cast(y_true, tf.float32),[-1])
    y_pred_f=tf.reshape(tf.dtypes.cast(y_pred, tf.float32),[-1])
    intersection=tf.reduce_sum(y_true_f*y_pred_f)
    return (2. * intersection + 1.) / (tf.reduce_sum(y_true_f) + tf.reduce_sum(y_pred_f) + 1.)

#https://stackoverflow.com/questions/49785133/keras-dice-coefficient-loss-function-is-negative-and-increasing-with-epochs
def dice_loss(y_true, y_pred):
    y_true_f = tf.reshape(y_true, [-1])
    y_pred_f = tf.reshape(y_pred, [-1])
    return (1-dice_coefficient(y_true, y_pred))

#defining function for calculation of loss function: binary cross entropy + dice loss
def bce_dice_loss(y_true, y_pred):
    y_true_f = tf.reshape(y_true, [-1])
    y_pred_f = tf.reshape(y_pred, [-1])
    return binary_crossentropy(y_true, y_pred) + (1-dice_coef(y_true, y_pred))

In [9]:
def sum_pixel(i):
    return sum([int(k) for k in i.split(' ')[1::2]])

In [10]:
#Reference: https://www.kaggle.com/paulorzp/rle-functions-run-lenght-encode-decode
def mask2rle(img):
    pixels= img.T.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

In [11]:
# Implementing custom data generator
#https://towardsdatascience.com/implementing-custom-data-generators-in-keras-de56f013581c
class test_DataGenerator(keras.utils.all_utils.Sequence):

  def __init__(self,dataframe,batch_size=1,shuffle=False,preprocess=None,info={}):
    self.batch_size = batch_size
    self.df = dataframe
    self.indices = self.df.index.tolist()
    self.preprocess = preprocess
    self.shuffle = shuffle
    self.on_epoch_end()

  def __len__(self):
    return len(self.indices) // (self.batch_size)

  def __getitem__(self, index):
    index = self.index[index * self.batch_size:(index + 1) * self.batch_size]
    batch = [self.indices[k] for k in index]

    X= self.__get_data(batch)
    return X

  def on_epoch_end(self):
    self.index = np.arange(len(self.indices))
    if self.shuffle == True:
      np.random.shuffle(self.index)

  def __get_data(self, batch):
    X = np.empty((self.batch_size,256,800,3),dtype=np.float32) # image place-holders


    for i, id in enumerate(batch):
      X[i,] = Image.open('/content/steel/test_images/' + str(self.df['image_id'].loc[id])).resize((800,256))


      # preprocess input
    if self.preprocess!=None: X = self.preprocess(X)

    return X

In [12]:
binary=load_model('/content/drive/MyDrive/binary_Xception_3.h5',custom_objects={'f1_score':f1_score})
multi=load_model('/content/drive/MyDrive/multi_label.h5',custom_objects={'f1_score':f1_score})
segment_1=load_model('/content/drive/MyDrive/segmentation_defect_1.h5',custom_objects={'bce_dice_loss':bce_dice_loss,'dice_coef':dice_coef})
# segment_2=load_model('/content/drive/MyDrive/segmentation_defect_2.h5',custom_objects={'bce_dice_loss':bce_dice_loss,'dice_coef':dice_coef})
# segment_3=load_model('/content/drive/MyDrive/segmentation_defect_3.h5',custom_objects={'bce_dice_loss':bce_dice_loss,'dice_coef':dice_coef})
# segment_4=load_model('/content/drive/MyDrive/segmentation_defect_4.h5',custom_objects={'bce_dice_loss':bce_dice_loss,'dice_coef':dice_coef})

TypeError: weight_decay is not a valid argument, kwargs should be empty  for `optimizer_experimental.Optimizer`.